In [10]:
import pandas as pd
import requests

# URL is only using central Sierra Nevada Monitoring station with Measurment 18 Daily Snow Depth, I beleive some Stations are missing due to not monitoring '18' Daily snow depth, there is a warning on the website that says "if the station doesn't use the measurment it will be dropped automatically"
url = "https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ALP%2CBLC%2CBLK%2CBLS%2CBSK%2CCAP%2CCSL%2CDAN%2CDDM%2CEBB%2CEP5%2CFDC%2CFLL%2CGIN%2CGKS%2CGNL%2CHGM%2CHOR%2CHRS%2CHVN%2CHYS%2CLBD%2CLVM%2CLVT%2CMDW%2CMNT%2CMRL%2CMSK%2CPSN%2CRBB%2CRBP%2CRP2%2CSDW%2CSIL%2CSLI%2CSPS%2CSPT%2CSTR%2CTCC%2CTNY%2CTUM%2CVRG%2CWC3%2CWHW&SensorNums=18&dur_code=D&Start=2013-01-01&End=2023-04-14"
response = requests.get(url).json()

In [17]:
station_list = []
date_list = []
value_list = []

for station in response:
    if "obsDate" in station:
        station_id = station['stationId']
        date = pd.to_datetime(station['obsDate'])
        value = station['value']
        station_list.append(station_id)
        date_list.append(date)
        value_list.append(value)

snow_df = pd.DataFrame({
    "Date" : date_list,
    "Station ID" : station_list,
    "Snow Depth (in)" : value_list,
})

snow_df["Month"] = snow_df["Date"].dt.month
snow_df["Year"] = snow_df["Date"].dt.year
snow_df['Snow Depth (in)'] = snow_df['Snow Depth (in)'].clip(lower = 0)
# snow_df['Snow Depth (in)'].min()

0

In [12]:
winter_bin = [0,4,10,12]
labels = ["Winter_End", "Summer", "Winter_Start"]

snow_binned = snow_df.copy()
snow_binned["Season"] = pd.cut(x = snow_binned["Month"], bins =  winter_bin, labels = labels)

snow_season = snow_binned.sort_values(["Year", "Month"])

season_list = []
working_year = 2014

for row in snow_season.iterrows():
    row_year = row[1][4]
    row_season = row[1][5]

    if (row_season == "Winter_End") & (row_year == working_year):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Winter_Start") & (row_year == (working_year - 1)):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Summer"):
        season_list.append(f"Summer {working_year - 1} {working_year}")
    else:
        season_list.append(f"na")

    if (row_year == (working_year + 1)):
        working_year = working_year + 1

snow_season["Season"] = season_list

In [13]:
winter_only_mask = snow_season["Season"].str.contains("Winter")
snow_season_winter_only = snow_season.loc[winter_only_mask]
snow_season_grouped = snow_season_winter_only.groupby(["Season"])
snow_season_grouped.mean().head(10)

,Snow Depth (in),Month,Year
Season,,,
Winter 2013 2014,25.142301,5.541180,2013.659349
Winter 2014 2015,23.408107,2.504352,2015.000000
Winter 2015 2016,55.120813,2.513636,2016.000000
Winter 2016 2017,94.233218,2.535291,2017.000000
Winter 2017 2018,-786.619014,2.471313,2018.000000
Winter 2018 2019,-30.982409,2.486196,2019.000000
Winter 2019 2020,-57.942179,2.462571,2020.000000
Winter 2020 2021,-534.569938,2.478900,2021.000000
Winter 2021 2022,42.992291,2.481571,2022.000000
